<a href="https://colab.research.google.com/github/tanishy7777/Pix2Pix-Pytorch/blob/main/Implementing_Pix2Pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/maps.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')

with zipfile.ZipFile('/content/drive/MyDrive/facades.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')

In [3]:
# Implement Mpap Dataset, Facades Dataset
# Implement Discriminator
# Implement Generator

In [4]:
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.optim import lr_scheduler

In [6]:
class Discriminator(nn.Module):
    """ PatchGAN Discriminator

    Here 70x70 Discriminator Architecture is used
    C64-C128-C256-C512

    After the last layer convolution is applied to map to a 1D prediction
    All ReLUs are Leaky with 0.2 slope
    BatchNorm is not applied to the first C64 layer

    Here Ck denote a Convolution-BatchNorm-ReLU layer
    with k filters.

    CDk denotes a Convolution-BatchNorm-Dropout-ReLU layer
    with a dropout rate of 50%.

    All convolutions are 4x4 spatial filters applied with stride 2.

    """
    def __init__(self, input_channels=3, hidden_units=[64, 128, 256, 512]):
        super().__init__()

        # Initial Block (dosent have BatchNorm as per the paper)
        sequence = [nn.Conv2d(input_channels, hidden_units[0], kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True)]

        for l in range(1, len(hidden_units)):  # gradually increase the number of filters
            sequence += [
                nn.Conv2d(hidden_units[l-1], hidden_units[l], kernel_size=4, stride=2, padding=1),
                nn.BatchNorm2d(hidden_units[l]),
                nn.LeakyReLU(0.2)
            ]

        sequence += [nn.Conv2d(hidden_units[-1], 1, kernel_size=4, stride=1, padding=1)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
      """Standard forward."""

      return self.model(input)

In [7]:
class UnetGenerator(nn.Module):
    """Create a Unet-based generator"""

    def __init__(self, input_nc, output_nc, num_downs, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet generator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            num_downs (int) -- the number of downsamplings in UNet. For example, # if |num_downs| == 7,
                                image of size 128x128 will become of size 1x1 # at the bottleneck
            ngf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer

        We construct the U-Net from the innermost layer to the outermost layer.
        It is a recursive process.
        """
        super(UnetGenerator, self).__init__()
        # construct unet structure
        unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=None, norm_layer=norm_layer, innermost=True)  # add the innermost layer
        for i in range(num_downs - 5):          # add intermediate layers with ngf * 8 filters
            unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer, use_dropout=use_dropout)
        # gradually reduce the number of filters from ngf * 8 to ngf
        unet_block = UnetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        self.model = UnetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True, norm_layer=norm_layer)  # add the outermost layer

    def forward(self, input):
        """Standard forward"""
        return self.model(input)


class UnetSkipConnectionBlock(nn.Module):
    """Defines the Unet submodule with skip connection.
        X -------------------identity----------------------
        |-- downsampling -- |submodule| -- upsampling --|

        Encoder:
        C64-C128-C256-C512-C512-C512-C512-C512

        Unet Decoder:
        CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128


        After the last layer in the decoder, a convolution is applied
        to map to the number of output channels followed by a Tanh
        function.

        BatchNorm is not applied to the first C64 layer in the encoder.

        All ReLUs in the encoder are leaky, with slope 0.2, while ReLUs
        in the decoder are not leaky.
    """

    def __init__(self, outer_channels, inner_channels, input_channels,
                 submodule=None, outermost=False, innermost=False, use_dropout=False):
        """Construct a Unet submodule with skip connections.

        Parameters:
            outer_nc (int) -- the number of filters in the outer conv layer
            inner_nc (int) -- the number of filters in the inner conv layer
            input_nc (int) -- the number of channels in input images/features
            submodule (UnetSkipConnectionBlock) -- previously defined submodules
            outermost (bool)    -- if this module is the outermost module
            innermost (bool)    -- if this module is the innermost module
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
        """
        super().__init__()
        self.outermost = outermost

        if input_nc is None:
            input_nc = outer_channels

        downconv = nn.Conv2d(input_channels, inner_channels, kernel_size=4, stride=2, padding=1)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = nn.BatchNorm2d(inner_channels)

        upnorm = nn.BatchNorm2d(outer_channels)
        uprelu = nn.ReLU(True)


        if outermost:
            upconv = nn.ConvTranspose2d(inner_channels * 2, outer_channels, kernel_size=4, stride=2, padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up

        elif innermost:
                                      # 512             512(same as no of filters)
            upconv = nn.ConvTranspose2d(inner_channels, outer_channels, kernel_size=4, stride=2, padding=1)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(inner_channels * 2, outer_channels, kernel_size=4, stride=2, padding=1)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = down + [submodule] + up + [nn.Dropout(0.5)]
            else:
                model = down + [submodule] + up

        self.model = nn.Sequential(*model)

    def forward(self, x):
        if self.outermost:
            return self.model(x)
        else:   # add skip connections
            return torch.cat([x, self.model(x)], 1)



## References

1. https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
2. https://github.com/aladdinpersson/Machine-Learning-Collection
3. https://phillipi.github.io/pix2pix/

In [ ]:
1.